In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r drive/MyDrive/IFT780/TP2/prog/* .

# TP2: Dropout

Dropout [1] est une technique de régularisation qui consiste à forcer aléatoirement à zéro certains neurones lors de la propagation avant. Pour cette démonstration, vous verrez comment est implémenté une couche de dropout et comment l'incorporer au réseau pleinement connecté codé lors du TP1.

Ceci est le notebook le plus court du tp2.  Nous utiliserons pour l'essentiel le code dans les fichiers suivants :

    model/Model.py
    model/Solver.py
    layers/Dropout.py 
    utils/model_loss.py
    
Les fichiers ci-dessus sont déjàs fournis et donnent une solution aux équipes ayant perdu des points dans ces parties.

[1] Geoffrey E. Hinton et al, "Improving neural networks by preventing co-adaptation of feature detectors", arXiv 2012


In [1]:
import random
import numpy as np
from utils.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Pour automatiquement recharger les modules externes
# voir http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Charger la banque de données CIFAR-10, prétraiter les images et ajouter une dimension pour le biais.
    
    Input :
    - num_training : nombre d'images à mettre dans l'ensemble d'entrainement
    - num_validation : nombre d'images à mettre dans l'ensemble de validation
    - num_test : nombre d'images à mettre dans l'ensemble de test
    - num_dev : d'images à mettre dans l'ensemble dev
    
    Output :
    - X_train, y_train : données et cibles d'entrainement
    - X_val, y_val: données et cibles de validation
    - X_test y_test: données et cibles de test 
    - X_dev, y_dev: données et cicles dev
    """
    # Charger les données CIFAR-10
    cifar10_dir = 'datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test, _ = load_CIFAR10(cifar10_dir)
  
    # Séparer en ensembles d'entraînement, de validation, de test et de dev
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalisation
    X_train -= np.mean(X_train, axis = 0)
    X_val -= np.mean(X_val, axis = 0)
    X_test -= np.mean(X_test, axis = 0)
    X_dev -= np.mean(X_dev, axis = 0)

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3072)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3072)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3072)
Test labels shape:  (1000,)
dev data shape:  (500, 3072)
dev labels shape:  (500,)


In [4]:
from layers.Dense import Dense
from layers.Dropout import Dropout
from model.Model import Model
from utils.model_loss import cross_entropy_loss

def create_toy_data(shape):
    np.random.seed(0)
    return np.random.randn(*shape) + 10

X = create_toy_data((500,500))

# Propagation avant

Dans le fichier `Dropout.py`, la propagation avant du dropout est implémentée. Puisque le dropout se comporte différemment en entraînement qu'en test, portez bien attention aux deux versions.

Exécutez la cellule que voici et assurez-vous que la moyenne de out_train soit la même que out_test.

NOTE : le "inverse dropout" est implémenté. Quelques ressources additionelles sont disponibles : 
- https://deepnotes.io/dropout
- https://cs231n.github.io/neural-networks-2/#reg

In [ ]:
for p in [0.3, 0.6, 0.75]:
    dropout_layer = Dropout(drop_rate=p)
    out_train = dropout_layer.forward(X, mode='train')
    out_test = dropout_layer.forward(X, mode='test')

    print('Running tests with p = ', p)
    # Les 3 moyennes que voici devraient être identiques.
    print('Mean of input: ', X.mean())
    print('Mean of train-time output: ', out_train.mean())
    print('Mean of test-time output: ', out_test.mean())
    print('Fraction of train-time output set to zero: ', (out_train == 0).mean())
    print('Fraction of test-time output set to zero: ', (out_test == 0).mean())


In [ ]:
from utils.gradients import evaluate_numerical_gradient

# Retourne l'erreur relative maximale des matrices de gradients passées en paramètre.
# Pour chaque paramètre, l'erreur relative devrait être inférieure à environ 1e-8.
def rel_error(x, y):
    rel = np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y)))
    return np.max(rel)

# Dropout: rétro-propagation
Toujours dans le fichier `Dropout.py`, la rétro-propagation du dropout est aussi implémentée.

In [ ]:
X = create_toy_data((10,10))
dA = np.random.randn(*X.shape)

dropout_layer = Dropout(drop_rate=0.8)

_ = dropout_layer.forward(X, mode='train')
dX = dropout_layer.backward(dA, mode='train')

drop_mask = dropout_layer.cache

# L'erreur relative devrait être très petite, inférieure à 1e-8
rel_error(dX, drop_mask * dA / 0.2)

# Réseau multi-couches avec Dropout
En principe, le code que voici devrait fonctionner.

In [ ]:
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D) / 5
y = np.random.randint(C, size=(N,))
seed = 42

for p in [0, 0.25, 0.5]:
    print('Running check with dropout = ', p)
    model = Model()
    
    dense1 = Dense(dim_input=D, dim_output=H1, weight_scale=5e-2, activation='relu')
    dropout1 = Dropout(drop_rate=p)
    
    dense2 = Dense(dim_input=H1, dim_output=H2, weight_scale=5e-2, activation='relu')
    dropout2 = Dropout(drop_rate=p)
    
    dense3 = Dense(dim_input=H2, dim_output=C, weight_scale=5e-2)
    dropout3 = Dropout(drop_rate=p)
    
    model.add(dense1, 'dense1')
    model.add(dropout1, 'dropout1')
    model.add(dense2, 'dense2')
    model.add(dropout2, 'dropout2')
    model.add(dense3, 'dense3')
    model.add(dropout3, 'dropout3')
    model.add_loss(cross_entropy_loss)
    
    out = model.forward(X, seed=seed)
    loss, dScores, _ = model.calculate_loss(out, y, 0.0)
    _ = model.backward(dScores)

    gradients = model.gradients()
    model_params = model.parameters()
    
    print('Initial loss: ', loss)
    
    # Les erreurs devraient être inférieures ou égales à 1e-5
    for layer_name, layer_params in model_params.items():
        for param_name, _ in layer_params.items():
            grad_num = evaluate_numerical_gradient(X, y, model, layer_name, param_name, reg=0.0, seed=seed)
            max_error = rel_error(grad_num, gradients[layer_name][param_name])

            print('%s max relative error: %e' % (layer_name + '-' + param_name, max_error))

# Expérimentation
Ici nous entrainerons 2 réseaux de neurones avec 500 données: l'un utilisera du dropout et l'autre non. Nous pourrons alors visualiser les justesses obtenues en entraînement et en validation.

In [ ]:
from model.Solver import epoch_solver, Adam

num_train = 500

X_train_small = X_train[:num_train]
y_train_small = y_train[:num_train]


train_accuracy_histories = []
val_accuracy_histories = []

dropouts = [0, 0.3]

for p in dropouts:
    model = Model()
    
    dense1 = Dense(dim_output=500, weight_scale=1e-2, activation='relu')
    dropout1 = Dropout(drop_rate=p)
    
    dense2 = Dense(dim_input=500, weight_scale=1e-2)
    dropout2 = Dropout(drop_rate=p)
    
    model.add(dense1, 'dense1')
    model.add(dropout1, 'dropout1')
    model.add(dense2, 'dense2')
    model.add(dropout2, 'dropout2')
    model.add_loss(cross_entropy_loss)
    
    print('\nDropout: ', p, '\n')
    
    optimizer = Adam(1e-4, model)
    
    _, train_accuracy_history, val_accuracy_history = epoch_solver(X_train_small, 
                                                                   y_train_small,
                                                                   X_val,
                                                                   y_val,
                                                                   0.0,
                                                                   optimizer,
                                                                   epochs=20)
    
    train_accuracy_histories.append(train_accuracy_history)
    val_accuracy_histories.append(val_accuracy_history)

In [ ]:
plt.subplot(3, 1, 1)
for i in range(2):
  plt.plot(train_accuracy_histories[i], '-', label='%.2f dropout' % dropouts[i])
plt.title('Train accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(ncol=2, loc='lower right')
  
plt.subplot(3, 1, 2)
for i in range(2):
  plt.plot(val_accuracy_histories[i], '-', label='%.2f dropout' % dropouts[i])
plt.title('Val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(ncol=2, loc='lower right')

plt.gcf().set_size_inches(15, 15)
plt.show()